In [ ]:
# 参照統合
options(warn=-1)
options(future.globals.maxSize = 100 * 1024^3)
# Pkgs <- c("Matrix", "tibble", "tidyr", "dplyr", "stringr", "Seurat", "SeuratWrappers", "DoubletFinder", "glmGamPoi", "magrittr", "ggplot2", "gplots", "RColorBrewer", "colorRamp2")
Pkgs <- c("Matrix", "tibble", "tidyr", "dplyr", "stringr", "Seurat", "hdf5r", "DoubletFinder", "glmGamPoi", "magrittr", "ggplot2", "gplots", "RColorBrewer")
for(p in Pkgs) suppressMessages(library(p, character.only=T))

# マルチプレット率(10X)
multiplet_rates_10x <- data.frame("Multiplet_rate"= c(0.004, 0.008, 0.0160, 0.023, 0.031, 0.039, 0.046, 0.054, 0.061, 0.069, 0.076),
                                                        "Loaded_cells" = c(800, 1600, 3200, 4800, 6400, 8000, 9600, 11200, 12800, 14400, 16000),
                                                        "Recovered_cells" = c(500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000))

# QC
Load_QC <- function(){
        # ファイル読込
        # x <- CreateSeuratObject(counts=Read10X_h5(paste0(DIR_PATH,dir,"/",dir,"-filtered_feature_bc_matrix.h5")), min.cells=5, min.features=5, project=dir)
        x <- CreateSeuratObject(counts=Read10X("data/"), min.cells=5, min.features=5)

        # 正規化＆次元削減
        x <- x %>%
                SCTransform(assay="RNA", verbose=FALSE) %>%
                FindVariableFeatures(nfeatures=3000) %>%
                RunPCA(npcs=200, verbose=FALSE)

        # PCs推定
        stdv <- x[["pca"]]@stdev
        percent_stdv <- (stdv/sum(stdv)) * 100
        cumulative <- cumsum(percent_stdv)
        co1 <- which(cumulative > 90 & percent_stdv < 5)[1] 
        co2 <- sort(which((percent_stdv[1:length(percent_stdv) - 1] - percent_stdv[2:length(percent_stdv)]) > 0.1), decreasing = T)[1] + 1
        min_pc <- min(co1, co2)

        # UMAP
        x <- x %>%
                FindNeighbors(dims=1:min_pc) %>%
                FindClusters(resolution=0.1) %>%
                RunUMAP(dims=1:min_pc)

        # pK値取得
        sweep_res_list <- paramSweep(x, PCs=1:min_pc, sct=TRUE)
        sweep_stats <- summarizeSweep(sweep_res_list, GT=FALSE)
        bcmvn <- find.pK(sweep_stats)
        optimal_pK <- as.numeric(as.character(as.character(bcmvn[which.max(bcmvn$BCmetric), "pK"])))

        # マルチプレット率推定
        multiplet_rate <- multiplet_rates_10x %>%
                dplyr::filter(Recovered_cells < nrow(x@meta.data)) %>% 
                dplyr::slice(which.max(Recovered_cells)) %>%
                dplyr::select(Multiplet_rate) %>%
                as.numeric(as.character())

        # マルチプレット数取得
        annotations <- x@meta.data$seurat_clusters
        homotypic_prop <- modelHomotypic(annotations)
        nExp_poi <- round(multiplet_rate * nrow(x@meta.data))
        nExp_poi_adj <- round(nExp_poi * (1 - homotypic_prop))

        if (!is.na(multiplet_rate))
        {
            x <- doubletFinder(x, PCs=1:min_pc, pN=multiplet_rate, pK=optimal_pK, nExp=nExp_poi_adj, reuse.pANN=FALSE, sct=TRUE)
            colnames(x@meta.data)[grepl('DF.classifications.*', colnames(x@meta.data))] <- "doublet_finder"
            x <- subset(x, doublet_finder == "Singlet")
            # x <- SubsetData(x, cells=rownames(x@meta.data)[which(x@meta.data$DF.classification == "Singlet")])
        }

        # 死細胞除去
        x[["percent.mt"]] <- PercentageFeatureSet(x, pattern = "^MT-")
        x <- subset(x, subset = percent.mt < 10)

        # 再処理
        # 正規化＆次元削減
        x <- x %>%
                SCTransform(assay="RNA", verbose=FALSE) %>%
                FindVariableFeatures(nfeatures=3000) %>%
                RunPCA(npcs=200, verbose=FALSE)
        
        # PCs推定
        stdv <- x[["pca"]]@stdev
        percent_stdv <- (stdv/sum(stdv)) * 100
        cumulative <- cumsum(percent_stdv)
        co1 <- which(cumulative > 90 & percent_stdv < 5)[1] 
        co2 <- sort(which((percent_stdv[1:length(percent_stdv) - 1] - percent_stdv[2:length(percent_stdv)]) > 0.1), decreasing = T)[1] + 1
        min_pc <- min(co1, co2)

        # UMAP
        x <- x %>%
                FindNeighbors(dims=1:min_pc) %>%
                FindClusters(resolution=0.1) %>%
                RunUMAP(dims=1:min_pc)

        return(x)
}
so <- Load_QC()
normalized_matrix <- as.matrix(GetAssayData(so, slot="data"))
write.table(normalized_matrix, file="data/expmat.tsv", sep="\t", quote=FALSE, row.names=TRUE, col.names=NA)
saveRDS(so, file="so.rds")

In [ ]:
##### SGCRNA_ALL #####
using CSV, DataFrames
using RCall
using JLD2
using SGCRNAs

# load data
Data = CSV.read("data/expmat.tsv", header=1, comment="#", delim='\t', DataFrame);

# calc correlation and gradient
CorData, GradData = SGCRNAs.CGM(Data.Column1, Matrix(Data[:,2:end]), fn="Result/");
# CorData = load_object("Result/_cor.jld2");
# GradData = load_object("Result/_grad.jld2");

# spectral clustering
clust, pos, edge_data = SGCRNAs.SpectralClustering(CorData, GradData);
# save_object("Result/_scdata.jld2", (clust, pos, edge_data));
# (clust, pos, edge_data) = load_object("Result/_scdata.jld2");

# set parameter
for i in 1:length(clust) println(maximum(clust[i])); end
for i in 1:maximum(clust[1]) println(i, "-", sum(clust[1] .== i)); end

# set parameter
d = 1; k = maximum(clust[d]);
GeneClust = DataFrame(Gene=names(edge_data), Module=clust[d]);
# edge_data |> CSV.write("Result/Score.tsv", delim='\t', writeheader=true)
GeneClust |> CSV.write("Result/GeneCluster.tsv", delim='\t', writeheader=true)

# draw network
nw, new_pos, cnctdf, new_clust, score = SGCRNAs.SetNetwork(edge_data, clust[d], pos, il=collect(1:k));
save_object("Result/_nwdata.jld2", (nw, new_pos, cnctdf, new_clust, score, edge_data))
# (nw, new_pos, cnctdf, new_clust, score, edge_data) = load_object("Result/_nwdata.jld2");
maximum(abs.(cnctdf.cor))
# DrawNetwork("Result/Fig/Visium_NegativeNetWork_All-0.0.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:N, edge_threshold=0.0, edge_scaler=50)
SGCRNAs.DrawNetwork("Result/Fig/Visium_AllNetWork-0.5.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:ALL, edge_threshold=0.5)
# cluster93
nw, new_pos, cnctdf, new_clust, score = SGCRNAs.SetNetwork(edge_data, clust[d], pos, il=[93]);
save_object("Result/93_nwdata.jld2", (nw, new_pos, cnctdf, new_clust, score, edge_data))
# (nw, new_pos, cnctdf, new_clust, score, edge_data) = load_object("Result/93_nwdata.jld2");
NodeLabel = unique(sort(vcat(cnctdf.e1,cnctdf.e2)))
# NodeLabel = [x in ["TP53","E2F4","EZH1","RARA","STAT5B","SUZ12","TBX2","FLCN"] ? x : "" for x in NodeLabel]
NodeLabel[sortperm(score, rev=true)[1:5]]
is = sortperm(score)[1:(length(score)-5)];
for i in is
    NodeLabel[i] = ""
end
SGCRNAs.DrawNetwork("Result/Fig/Visium_93NetWork_Top5.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:ALL, edge_threshold=0.0, node_labels=NodeLabel)
# cluster69
nw, new_pos, cnctdf, new_clust, score = SGCRNAs.SetNetwork(edge_data, clust[d], pos, il=[69]);
save_object("Result/69_nwdata.jld2", (nw, new_pos, cnctdf, new_clust, score, edge_data))
# (nw, new_pos, cnctdf, new_clust, score, edge_data) = load_object("Result/69_nwdata.jld2");
NodeLabel = unique(sort(vcat(cnctdf.e1,cnctdf.e2)));
# NodeLabel = [x in ["HES1","PPARG","BHLHE40","MITF","NR1D2"] ? x : "" for x in NodeLabel]
NodeLabel[sortperm(score, rev=true)[1:5]]
is = sortperm(score)[1:(length(score)-5)]
for i in is
    NodeLabel[i] = ""
end
SGCRNAs.DrawNetwork("Result/Fig/Visium_69NetWork_Top5.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:ALL, edge_threshold=0.0, node_labels=NodeLabel)

# Tumor
IDlist = [7, 10, 25, 30, 44, 58, 63, 69, 87, 93, 97];
for i in IDlist
    nw, new_pos, cnctdf, new_clust, score = SGCRNAs.SetNetwork(edge_data, clust[d], pos, il=[i]);
    # save_object("Result/"*string(i)*"_nwdata.jld2", (nw, new_pos, cnctdf, new_clust, score, edge_data))
    # (nw, new_pos, cnctdf, new_clust, score, edge_data) = load_object("Result/"*string(i)*"_nwdata.jld2");
    NodeLabel = unique(sort(vcat(cnctdf.e1,cnctdf.e2)));
    # ranks = sortperm(score)[1:(length(score)-5)]
    # for r in ranks
    #     NodeLabel[r] = ""
    # end
    # SGCRNAs.DrawNetwork("Result/Fig/Visium_NetWork-"*string(i)*".png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:ALL, edge_threshold=0.0, node_labels=NodeLabel)
    println(i, "-", (length(score) > 4 ? NodeLabel[sortperm(score, rev=true)[1:5]] : NodeLabel))
end

# GO analysis
ro = RObject(edge_data);
fl = readdir("Result/Fig/");
rm.("Result/Fig/" .* fl[occursin.("GO_cluser",fl)]);
R"""
    Pkgs <- c("clusterProfiler", "enrichplot", "org.Hs.eg.db", "ggplot2")
    for(p in Pkgs) suppressMessages(library(p, character.only=T))

    options(ggrepel.max.overlaps = Inf)

    Analysis_GO <- function(x) {
        result <- enrichGO(
                            gene=x,
                            keyType="SYMBOL",
                            OrgDb=org.Hs.eg.db,
                            ont="ALL",                #"BP","CC","MF","ALL"
                            pAdjustMethod="BH",
                            pvalueCutoff=0.05,
                            qvalueCutoff=0.05
                        )
    }
    PlotFig <- function(res, i) {
        res_simple <- simplify(res)
        Fig <- clusterProfiler::dotplot(res_simple)
        ggsave(paste0("Result/Fig/Visium_GO_cluser-",i,"_dot_simple.png"), plot=Fig, dpi=400)
        # Fig <- clusterProfiler::dotplot(res)
        # ggsave(paste0("Result/Fig/Visium_GO_cluser-",i,"_dot_all.png"), plot=Fig, dpi=400)
    }

    for(i in 1:$k)
    {
        ed <- $ro
        nc <- $new_clust
        commun = colnames(ed)[which(nc == i)]
        result <- try(Analysis_GO(commun), silent=FALSE)
        if (class(result) != "try-error") {
            y <- try(PlotFig(result, i), silent=FALSE)
        }
    }
"""



In [ ]:
# Draw spatial expression heatmap of each module (SGCRNA)
using JSON
using Images, FileIO
using Colors
using CSV, DataFrames

# データ読み込み
Data = CSV.read("Result/GeneCluster.tsv", header=true, comment="#", delim='\t', DataFrame);
ExpDF = CSV.read("data/expmat.tsv", header=true, comment="#", delim='\t', DataFrame);
rename!(ExpDF, :Column1 => :Gene);
ExpDF = innerjoin(Data[:,[1]], ExpDF, on=:Gene);

ModuleNum = maximum(Data.Module);
ScaleFactor = JSON.parsefile("data/scalefactors_json.json");
r = ScaleFactor["spot_diameter_fullres"] * ScaleFactor["tissue_hires_scalef"] / 2.0;
BarcodePos = CSV.read("data/tissue_positions_list.csv", header=1, comment="#", delim=',', DataFrame);
BarcodePos.pxl_row_in_fullres .*= ScaleFactor["tissue_hires_scalef"];
BarcodePos.pxl_col_in_fullres .*= ScaleFactor["tissue_hires_scalef"];
BarcodePos[!, :RowMin] = BarcodePos.pxl_row_in_fullres .- r;
BarcodePos[!, :RowMax] = BarcodePos.pxl_row_in_fullres .+ r;
BarcodePos[!, :ColMin] = BarcodePos.pxl_col_in_fullres .- r;
BarcodePos[!, :ColMax] = BarcodePos.pxl_col_in_fullres .+ r;
rename!(BarcodePos, 1 => "Barcode");

imgbuf = load("data/tissue_hires_image.png");
PosList = DataFrame(NamedTuple{(:Row, :Col)}.(vec(collect(Base.product(1:size(imgbuf,1), 1:size(imgbuf,2))))));

for c in 76:ModuleNum
    img = deepcopy(imgbuf);
    ModuleExp = innerjoin(Data[Data.Module .== c, [:Gene]], ExpDF, on=:Gene)[:, 2:end];
    ModuleExp = DataFrame(hcat(names(ModuleExp),map(sum, eachcol(ModuleExp))), [:Barcode,:Exp]);
    ModuleExp.Exp .-= (minimum(ModuleExp.Exp) - 1);
    ModuleExp.Exp = round.(Int, ModuleExp.Exp);
    Pos = innerjoin(BarcodePos, ModuleExp[:,[:Barcode]], on=:Barcode, order=:right);
    Colour = range(RGBA(0.0,0.0,0.0,1.0), stop=RGBA(1.0,1.0,0.0,1.0), length=Int(ceil(maximum(ModuleExp.Exp))));
    for p in 1:nrow(Pos)
        Q = (Pos.RowMin[p] .< PosList.Row) .& (PosList.Row .< Pos.RowMax[p]) .& (Pos.ColMin[p] .< PosList.Col) .& (PosList.Col .< Pos.ColMax[p])
        SpotPos = PosList[Q, :]
        Q = ((SpotPos.Row .- Pos.pxl_row_in_fullres[p]).^2 + (SpotPos.Col .- Pos.pxl_col_in_fullres[p]).^2) .< r^2
        SpotPos = SpotPos[Q, :]
        for s in 1:nrow(SpotPos)
            img[SpotPos.Row[s], SpotPos.Col[s]] = Colour[ModuleExp.Exp[p]]
        end
    end
    save("Result/Fig/Visium_SGCRNA_Clust-"*string(c)*".png", img)
    img = 0; ModuleExp = 0; Pos = 0; Colour = 0; GC.gc()
    println(c)
    sleep(5)
end


# hdWGCNA

In [ ]:
options(warn=-1)
Pkgs <- c("Seurat", "WGCNA", "hdWGCNA")
for(p in Pkgs) suppressMessages(library(p, character.only=T))

set.seed(42)

# 読込
tissue_positions <- read.csv("data/tissue_positions_list.csv")
so <- readRDS("so.rds")

# 前処理
so$barcode <- colnames(so)
tissue_positions <- subset(tissue_positions, barcode %in% so$barcode)
new_meta <- dplyr::left_join(so@meta.data, tissue_positions, by='barcode')
so$row <- new_meta$array_row 
so$imagerow <- new_meta$pxl_row_in_fullres
so$col <- new_meta$array_col
so$imagecol <- new_meta$pxl_col_in_fullres

# Construct metaspots
so <- SetupForWGCNA(so, gene_select="fraction", fraction=0.05, wgcna_name="vis")
# so <- MetaspotsByGroups(so, group.by=c("region"), ident.group="region", assay="SCT")
# so <- NormalizeMetacells(so)

# Co-expression network analysis
so  <- SetDatExpr(so, group.by=NULL, group_name=NULL)
so <- TestSoftPowers(so)
# plot_list <- PlotSoftPowers(so)
# wrap_plots(plot_list, ncol=2)
so <- ConstructNetwork(so, tom_name="test", overwrite_tom=TRUE)
# PlotDendrogram(so, main='Spatial hdWGCNA dendrogram')

# so <- ModuleEigengenes(so)
# so <- ModuleConnectivity(so)
modules <- GetModules(so)

write.table(modules, "Result/hdWGCNA.tsv", sep='\t', row.names=FALSE, col.names=TRUE, quote=FALSE)

# SGCP

In [ ]:
options(warn=-1)
options(future.globals.maxSize = 100 * 1024^3)
Pkgs <- c("SGCP", "org.Hs.eg.db", "AnnotationDbi")
for(p in Pkgs) suppressMessages(library(p, character.only=T))

Data <- read.table("data/expmat.tsv", header=TRUE);
genes <- AnnotationDbi::select(org.Hs.eg.db, keys=row.names(Data), columns=c("ENTREZID"), keytype="SYMBOL")
genes <- genes[(!is.na(genes$SYMBOL)) & (!is.na(genes$ENTREZID)), ]
genes <- genes[(!duplicated(genes$SYMBOL)) & (!duplicated(genes$ENTREZID)),]
Data$SYMBOL <- row.names(Data)
Data <- merge(genes, Data, by="SYMBOL")
genes <- Data[, 1:2]
Data <- Data[, -c(1:2)]
vars <- apply(Data, 1, var)
zeroInd <- which(vars == 0)
if(length(zeroInd) != 0) {
        Data <- Data[-zeroInd, ]
        genes <- genes[-zeroInd, ]
}
xx <- as.list(org.Hs.egGO[genes$ENTREZID])
haveGO <- sapply(xx, function(x) {if (length(x) == 1 && is.na(x)) FALSE else TRUE })
numNoGO <- sum(!haveGO)
if(numNoGO != 0){
        Data <- Data[haveGO, ]
        genes <- genes[haveGO, ]
}
rownames(Data) <- genes$ENTREZID
sgcp <- ezSGCP(expData=as.matrix(Data), geneID=genes$ENTREZID, annotation_db="org.Hs.eg.db", eff.egs=FALSE , saveOrig=FALSE, sil=TRUE, hm=NULL)
names(genes) <- c("Symbol","geneID")
res <- merge(genes, sgcp$clusterLabels, by="geneID")
write.table(res[,c(2,4)], "Result/SGCP.tsv", sep='\t', row.names=FALSE, col.names=NA)


In [ ]:
# sankey
using CSV, DataFrames
using SankeyMakie
using CairoMakie

SGCRNA = CSV.read("Result/GeneCluster.tsv", header=1, comment="#", delim='\t', DataFrame);
WGCNA = CSV.read("Result/hdWGCNA.tsv", header=1, comment="#", delim='\t', DataFrame);
rename!(WGCNA, 1 => "Gene");
WGCNA = WGCNA[:, Not(2)];
Data = outerjoin(SGCRNA, WGCNA, on=:Gene);
Data = coalesce.(Data, "NA");
Data[!, :Module] = string.(Data.Module);
gdf = groupby(Data[:,[:Module ,:color]], [:Module ,:color]);
Data = combine(gdf, nrow);
sort!(Data);
Label = vcat(unique(Data.Module), unique(sort(Data.color)));

src = [findfirst(x -> x == i, Label) for i in Data.Module];
dst = [findfirst(x -> x == i, Label) for i in Data.color];

cnct = [(src[i], dst[i], Data.nrow[i]) for i in 1:length(src)];
f, ax, s = sankey(cnct, nodelabels=Label, figure = (; resolution = (1000, 2000)))
hidedecorations!(ax)
hidespines!(ax)
save("Result/Fig/Visium_Sankey.png", f)


In [ ]:
# Draw spatial expression heatmap of each module (All) (hdWGCNA)
using JSON
using Images, FileIO
using Colors
using CSV, DataFrames

# データ読み込み
ScaleFactor = JSON.parsefile("data/scalefactors_json.json");
r = ScaleFactor["spot_diameter_fullres"] * ScaleFactor["tissue_hires_scalef"] / 2.0;
BarcodePos = CSV.read("data/tissue_positions_list.csv", header=1, comment="#", delim=',', DataFrame);
BarcodePos.pxl_row_in_fullres .*= ScaleFactor["tissue_hires_scalef"];
BarcodePos.pxl_col_in_fullres .*= ScaleFactor["tissue_hires_scalef"];
BarcodePos[!, :RowMin] = BarcodePos.pxl_row_in_fullres .- r;
BarcodePos[!, :RowMax] = BarcodePos.pxl_row_in_fullres .+ r;
BarcodePos[!, :ColMin] = BarcodePos.pxl_col_in_fullres .- r;
BarcodePos[!, :ColMax] = BarcodePos.pxl_col_in_fullres .+ r;
rename!(BarcodePos, 1 => "Barcode");

imgbuf = load("data/tissue_hires_image.png");
PosList = DataFrame(NamedTuple{(:Row, :Col)}.(vec(collect(Base.product(1:size(imgbuf,1), 1:size(imgbuf,2))))));

ExpDF = CSV.read("Result/ExpDF.tsv", header=true, comment="#", delim='\t', DataFrame);
Data = CSV.read("Result/hdWGCNA.tsv", header=true, comment="#", delim='\t', DataFrame);
rename!(Data, 1 => :Gene)
ModuleColour = unique(Data.module);

for c in ModuleColour
    img = deepcopy(imgbuf)
    ModuleExp = innerjoin(Data[Data.module .== c, [:Gene]], ExpDF, on=:Gene)[:, 2:end]
    ModuleExp = DataFrame(hcat(names(ModuleExp),map(sum, eachcol(ModuleExp))), [:Barcode,:Exp])
    ModuleExp.Exp .-= (minimum(ModuleExp.Exp) - 1)
    Pos = innerjoin(BarcodePos, ModuleExp[:,[:Barcode]], on=:Barcode, order=:right)
    Colour = range(RGBA(0.0,0.0,0.0,1.0), stop=RGBA(1.0,1.0,0.0,1.0), length=maximum(ModuleExp.Exp));
    for p in 1:nrow(Pos)
        Q = (Pos.RowMin[p] .< PosList.Row) .& (PosList.Row .< Pos.RowMax[p]) .& (Pos.ColMin[p] .< PosList.Col) .& (PosList.Col .< Pos.ColMax[p])
        SpotPos = PosList[Q, :]
        Q = ((SpotPos.Row .- Pos.pxl_row_in_fullres[p]).^2 + (SpotPos.Col .- Pos.pxl_col_in_fullres[p]).^2) .< r^2
        SpotPos = SpotPos[Q, :]
        for s in 1:nrow(SpotPos)
            img[SpotPos.Row[s], SpotPos.Col[s]] = Colour[ModuleExp.Exp[p]]
        end
    end
    save("Result/Fig/Visium_hdWGCNA_Clust-"*string(c)*".png", img)
end
